In [1]:
import findspark
findspark.init('/home/nowshad/spark-3.3.2-bin-hadoop3/')
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('lr-example').getOrCreate()

23/03/19 02:21:30 WARN Utils: Your hostname, nowshad resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/19 02:21:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 02:21:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/19 02:21:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data=spark.read.csv('/home/nowshad/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv',inferSchema=True, header=True)

In [5]:
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [6]:
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [8]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [11]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [15]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='indexed_cruise')

In [18]:
output=indexer.fit(data).transform(data)

In [24]:
output.createOrReplaceTempView('output')
spark.sql('Select distinct Cruise_line, indexed_cruise from output').show()

+-----------------+--------------+
|      Cruise_line|indexed_cruise|
+-----------------+--------------+
|            Costa|           5.0|
|        Norwegian|           4.0|
|              MSC|           7.0|
|           Orient|          19.0|
|Regent_Seven_Seas|          10.0|
|           Disney|          18.0|
|         Windstar|          15.0|
|              P&O|           8.0|
|  Royal_Caribbean|           0.0|
|         Seabourn|          14.0|
|             Star|           9.0|
|         Princess|           2.0|
|          Oceania|          13.0|
|          Azamara|          16.0|
| Holland_American|           3.0|
|           Cunard|          12.0|
|        Celebrity|           6.0|
|        Silversea|          11.0|
|          Crystal|          17.0|
|         Carnival|           1.0|
+-----------------+--------------+



In [25]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- indexed_cruise: double (nullable = false)



In [26]:
output.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'indexed_cruise']

In [27]:
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','crew','indexed_cruise'],outputCol='features')

In [28]:
output1=assembler.transform(output)

In [29]:
output1.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- indexed_cruise: double (nullable = false)
 |-- features: vector (nullable = true)



In [32]:
for item in output1.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55
16.0
[6.0,30.276999999999997,6.94,5.94,3.55,42.64,3.55,16.0]


In [33]:
final_data=output1.select('features','crew')

In [34]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [38]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [35]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [36]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               110|
|   mean| 7.654272727272739|
| stddev|3.5606943089283423|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [37]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               48|
|   mean|8.114791666666665|
| stddev|3.383423917052409|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [39]:
lr=LinearRegression(labelCol='crew')

In [40]:
lr_model=lr.fit(train_data)

23/03/19 03:06:08 WARN Instrumentation: [b619f504] regParam is zero, which might cause numerical instability and overfitting.


In [49]:
lr_model.coefficients

DenseVector([0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 1.0, -0.0])

In [41]:
test_results=lr_model.evaluate(test_data)

In [42]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|7.993605777301127...|
|7.993605777301127...|
|1.065814103640150...|
|-1.77635683940025...|
|7.105427357601002...|
|3.552713678800501...|
|2.664535259100375...|
|-3.55271367880050...|
|7.105427357601002...|
|-8.88178419700125...|
|-8.88178419700125...|
|-1.42108547152020...|
|1.776356839400250...|
|3.552713678800501...|
|5.329070518200751...|
|3.552713678800501...|
|                 0.0|
|-7.10542735760100...|
|-7.10542735760100...|
|-7.10542735760100...|
+--------------------+
only showing top 20 rows



In [43]:
test_results.rootMeanSquaredError

9.245801214220991e-15

In [44]:
test_results.r2

1.0

In [45]:
unlabeled_data=test_data.select('features')

In [46]:
predictions=lr_model.transform(unlabeled_data)

In [47]:
predictions.head(1)

[Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 3.55, 16.0]), prediction=3.549999999999992)]

In [48]:
test_data.head(1)

[Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 3.55, 16.0]), crew=3.55)]